In [0]:
## mount files:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os, os.path
folder = 'Moocs/coursera_Duke/notebooks'
os.chdir('/content/drive/My Drive/'+folder)

In [0]:
# simple jpeg compression

import os
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image
from skimage import color
from scipy.fftpack import dct, idct

# required to display matplotlib plots in notebooks
%matplotlib inline

def read_image(img_path):
    return np.array(Image.open(img_path)).astype('float64')

def normalize_img(np_img): # range: 0 -> 1
    # the formula is: (x - min(x)) / (max(x) - min(x))
    if (np.max(np_img) == np.min(np_img)):
        return False
    return (np_img - np.min(np_img)) / (np.max(np_img) - np.min(np_img))

def show_img(np_img, is_grayscale):
    if is_grayscale:
        plt.imshow(np_img, cmap=plt.get_cmap('gray'))
    else:
        plt.imshow(np_img)
    plt.show()

def ycbcr_rgb_conversion(img, dest_channels):
    # img values should be between [0..255]
    if (dest_channels == 'RGB'):
        conversion_matrix = np.array([[1, 0, 1.402], 
                                      [1, -0.34414, -0.71414], 
                                      [1, 1.772, 0]]).astype('float64')
        img[:, :, [1,2]] -= 128
        rgb = img.dot(conversion_matrix.T)
        return rgb
    elif (dest_channels == 'YCbCr'):
        conversion_matrix = np.array([[0.299, -0.1689, 0.4998],
                                      [0.587, -0.3316, -0.4185],
                                      [0.114, 0.5005, -0.0812]]).astype('float64')
        ycbcr = img.dot(conversion_matrix)
        ycbcr[:, : , [1,2]] += 128
        return ycbcr
    return False

def convert_img_colors(dest_color, np_img):
    if dest_color == 'RGB':
        return color.gray2rgb(np_img)
    elif dest_color == 'GS':
        return color.rgb2gray(np_img)
    else:
        return False

def padding_img(img, block_size):
     # if the image shape is not divided by block_size then padding zeros ((top, bottom), (left, right))
    right_padding = 0
    bottom_padding = 0
    rows_modulu = img.shape[0] % block_size 
    cols_modulu = img.shape[1] % block_size
    if (rows_modulu != 0):
        bottom_padding += rows_modulu + (block_size - rows_modulu) - 1
    if (cols_modulu != 0):
        right_padding += cols_modulu + (block_size - cols_modulu) - 1
    return np.pad(img, ((0, bottom_padding), (0, right_padding)), 'constant', constant_values=0)

def divide_blocks(img, block_size=8):
    pad_img = padding_img(img, block_size)
    img_slices = np.zeros((int(pad_img.shape[0] / block_size), int(pad_img.shape[1] / block_size)), dtype=object)
    for i in range(img_slices.shape[0]):
        for j in range(img_slices.shape[1]):
            img_slices[i][j] = pad_img[(i*block_size):((i+1)*block_size), (j*block_size):((j+1)*block_size)]
    return img_slices

def compute_transform(img_slices, transform_name='DCT'):
    img_slices_transformed = np.zeros(img_slices.shape, dtype=object)
    for i in range(img_slices_transformed.shape[0]):
        for j in range(img_slices_transformed.shape[1]):
            img_slices_transformed[i][j] = dct(dct(img_slices[i][j], axis=0), axis=1)
    return img_slices_transformed

def quantize_helper(desired_intensity_levels, img):
    # desired_intensity_levels should be 2^k for k in [1..8] and the image should be grayscale
    divisor = 256 / desired_intensity_levels
    return np.floor(img / divisor) * divisor

def quantize_blocks(img_slices_transformed, quant_number):
    img_slices_quantized = np.zeros(img_slices_transformed.shape, dtype=object)
    for i in range(img_slices_quantized.shape[0]):
        for j in range(img_slices_quantized.shape[1]):
            img_slices_quantized[i][j] = quantize_helper(quant_number, img_slices_transformed[i][j])
    return img_slices_quantized

def get_k_largest_indexes(arr, k):
    sort_descending_arr = np.sort(arr.flatten())
    unique_vals = np.unique(sort_descending_arr)[::-1]
    max_k_vals = unique_vals[0:k]
    max_idx = np.isin(arr, max_k_vals).astype(int)
    return np.argwhere(max_idx==1)

def assign_largest_numbers(img_quantized_slice, img_largest_idx):
    tmp_image = np.zeros(img_quantized_slice.shape) # init each block with zeros
    for idx_arr in img_largest_idx:
        tmp_image[tuple(idx_arr)] = img_quantized_slice[tuple(idx_arr)]
    return tmp_image

def preserve_largest_coeff(img_slices_quantized, num_of_coeff=8):
    img_slices_coeff = np.zeros(img_slices_quantized.shape, dtype=object)
    img_largest_idx = np.zeros(img_slices_quantized.shape, dtype=object)
    for i in range(img_largest_idx.shape[0]): # get largest numbers idxs
        for j in range(img_largest_idx.shape[1]):
            img_largest_idx[i][j] = get_k_largest_indexes(img_slices_quantized[i][j], num_of_coeff)
    for i in range(img_slices_coeff.shape[0]): # assign the largest numbers
        for j in range(img_slices_coeff.shape[1]):
            img_slices_coeff[i][j] = assign_largest_numbers(img_slices_quantized[i][j], img_largest_idx[i][j])
    return img_slices_coeff

def compute_inv_transform(img_slices_final, original_transform='DCT'):
    img_slices_inv_transformed = np.zeros(img_slices_final.shape, dtype=object)
    for i in range(img_slices_inv_transformed.shape[0]):
        for j in range(img_slices_inv_transformed.shape[1]):
            img_slices_inv_transformed[i][j] = idct(idct(img_slices_final[i][j], axis=0), axis=1)
    return img_slices_inv_transformed

def slices_to_image(slices, block_size=8):
    final_image = np.zeros((int(block_size*slices.shape[0]), int(block_size*slices.shape[1])))   
    for i in range(0, slices.shape[0]):
        for j in range(0, slices.shape[1]):
            final_image[(i*block_size):((i+1)*block_size), (j*block_size):((j+1)*block_size)] = slices[i][j]
    return final_image

def reconstruct_image(img_slices_final, original_dim, original_transform='DCT', block_size=8):
    img_slices_inv_transformed = compute_inv_transform(img_slices_final, original_transform='DCT')
    final_image = slices_to_image(img_slices_inv_transformed, block_size)
    return final_image[0:original_dim[0], 0:original_dim[1]]
    
def jpeg_compress(img):
    # img should be one channel image
    block_size=8
    img_slices = divide_blocks(img, block_size=block_size)
    img_slices_transformed = compute_transform(img_slices, transform_name='DCT')
    img_slices_quantized = quantize_blocks(img_slices_transformed, quant_number=2)
    img_slices_coeff = preserve_largest_coeff(img_slices_quantized, num_of_coeff=50)
    return reconstruct_image(img_slices_coeff, img.shape, original_transform='DCT', block_size=block_size)

def jpeg_compress_rgb(img):
    # jpeg compression for 3 channels
    ycbcr_img = ycbcr_rgb_conversion(img, 'YCbCr')
    Y_jpg = jpeg_compress(ycbcr_img[:, :, 0])
    Cb_jpg = jpeg_compress(ycbcr_img[:, :, 1])
    Cr_jpg = jpeg_compress(ycbcr_img[:, :, 2])
    ycbcr_img[:, :, 0] = Y_jpg
    ycbcr_img[:, :, 1] = Cb_jpg
    ycbcr_img[:, :, 2] = Cr_jpg
    ycbcr_img = (255 * normalize_img(ycbcr_img)).astype('int') # normalize to [0..255]
    return ycbcr_rgb_conversion(ycbcr_img, 'RGB')

def get_random_image():
    import requests
    from io import BytesIO
    height=600
    width=600
    url='https://picsum.photos/'+str(height)+'/'+str(width)+'.jpg'
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return np.array(img).astype('float64')
    
def save_img(img, path):
    if (len(img.shape) == 2):
        img = convert_img_colors('RGB', img)
    if (np.max(img) > 1.0):
        img = normalize_img(img)
    image.imsave(path + ".jpg", img)
    print('image saved')

def main():
    # main_path = './data'
    # image_name = 'vietnam.jpg'
    # img = read_image(os.path.join(main_path, image_name))
    img = get_random_image()
    
    print('original rgb')
    src_img = normalize_img(img)
    show_img(src_img, False)
    
    print('compressed rgb')
    rgb_jpg_img = jpeg_compress_rgb(img)
    rgb_jpg_img = normalize_img(rgb_jpg_img)
    show_img(rgb_jpg_img, False)

    print('error rgb')
    show_img(normalize_img(normalize_img(img)-rgb_jpg_img), False)
    
    print('original grayscale')
    gs_img = convert_img_colors('GS', img)
    gs_img_norm = normalize_img(gs_img)
    show_img(gs_img_norm, True)

    print('compressed grayscale')
    gs_jpg_img = jpeg_compress(gs_img)
    gs_jpg_img = normalize_img(gs_jpg_img)
    show_img(gs_jpg_img, True)

    print('error gs')
    show_img(normalize_img(gs_img_norm-gs_jpg_img), True)

    # jpg_img_name = image_name.split('.jpg')[0] + '_compressed'
    # save_img(rgb_jpg_img, os.path.join(main_path, jpg_img_name))

main()